# 제4장 비정형 데이터 마이닝

## 제 2절 사회연결망 분석

### 1. 사회연결망 분석 정의
#### 가. SNA 정의

- 사회연결망 분석( Social network analysis )은 개인과 집단들 간의 관계를 노드와 링크로서 모델링해 그것의 위상구조와 확산 및 진화과정을 계량적으로 분석하는 방법론 
- 사화연결망 분석방법에는 
    - 집합론적인 방법
    - 그래프 이론에 의한 방법
    - 행렬을 이용한 방법

##### 1) 집합론적 방법

- 객체들의 집합에서 각 객체들 간의 관계를 다음과 같이 관계 쌍을 표현

##### 2) 그래프 이론을 이용한 방법 

![](data_03.jpg)

##### 3) 행렬을 이용한 방법

![](data_04.jpg)